In [9]:
import pandas as pd
reward = {"1": 10, "2":20, "3": 40}
all_rewards = [reward for _ in range(5)]
all_rewards

[{'1': 10, '2': 20, '3': 40},
 {'1': 10, '2': 20, '3': 40},
 {'1': 10, '2': 20, '3': 40},
 {'1': 10, '2': 20, '3': 40},
 {'1': 10, '2': 20, '3': 40}]

In [8]:
pd.DataFrame(all_rewards).sum().mean()

116.66666666666667

In [20]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr 11 12:11:49 2024

@author: naftabi
"""

import torch
import torch.nn as nn
import torch.optim as optim

import random

from utils import Memory
from model import QNetwork

class DQLAgent:
    def __init__(self, 
                 state_dim: int, 
                 action_dim: int, 
                 hidden_dim: int = 64, 
                 lr: float = 1e-4, 
                 gamma: float = 0.99,
                 tau: float = 0.05,
                 update_every: int = 20,
                 epsilon_start: float = 0.9,
                 epsilon_end: float = 0.05,
                 epsilon_decay: float = 0.995,
                 buffer_size: int = 10000,
                 seed: int = None):
        
        if seed is not None:
            random.seed(seed)
            torch.manual_seed(seed)
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
        
        self.gamma = gamma
        self.tau = tau
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.step_count = 0
        self.update_every = update_every
        self.epsilon = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay = epsilon_decay
        
        # Networks
        self.q_network = QNetwork(state_dim, action_dim, hidden_dim)
        self.target_q_network = QNetwork(state_dim, action_dim, hidden_dim)
        self.target_q_network.load_state_dict(self.q_network.state_dict())
        self.target_q_network.eval()
        
        self.optimizer = optim.RMSprop(self.q_network.parameters(), lr=lr)
        self.memory = Memory(buffer_size)
        self.criterion = nn.MSELoss()
        
    def update_epsilon(self):
        # Decay epsilon
        self.epsilon = max(self.epsilon_end, self.epsilon_decay * self.epsilon)  
        
    def act(self, state):
        self.q_network.eval()
        if random.random() > self.epsilon:
            state = torch.FloatTensor(state).unsqueeze(0)
            with torch.no_grad():
                q_value = self.q_network.forward(state)
            action = q_value.max(1)[1].item()
        else:
            action = random.randrange(self.action_dim)
        self.update_epsilon()
        return action
    
    def update(self, batch_size):
        self.q_network.train()
        state, action, reward, next_state = self.memory.sample(batch_size)
        state = torch.FloatTensor(state)
        next_state = torch.FloatTensor(next_state)
        action = torch.LongTensor(action)
        reward = torch.FloatTensor(reward)
        
        q_values = self.q_network(state)
        with torch.no_grad():
            next_q_values = self.q_network(next_state)
        
        q_value = q_values.gather(1, action.unsqueeze(1)).squeeze(1)
        max_next_q_value = next_q_values.max(1)[0]
        expected_q_value = reward + self.gamma * max_next_q_value 
        
        loss = self.criterion(q_value, expected_q_value)
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        self.step_count += 1
        if self.step_count % self.update_every == 0:
            for target_params, params in zip(self.target_q_network.parameters(), self.q_network.parameters()):
                target_params.data.copy_(self.tau * params.data + (1.0 - self.tau) * target_params.data)
                
    def load_policy(self, idx):
        filepath = f'./saved_q/agent_{idx}.pkl'
        try:
            state_dict = torch.load(filepath, map_location=torch.device('cpu'))
            self.q_network.load_state_dict(state_dict)
        except FileNotFoundError:
            raise Exception(f"Error: Policy file not found at {filepath}")
            

agent = DQLAgent(16, 4, hidden_dim=128)
agent.load_policy("i_cr30_101")

Policy loaded successfully from ./saved_q/agent_i_cr30_101.pkl


In [5]:
state_dict = torch.load("./saved_q/agent_i_cr30_101.pkl", map_location=torch.device('cpu'))

In [16]:
state_dict["fc1.weight"]

torch.Size([128, 16])

In [34]:
from math import pi, sqrt
import numpy as np
def get_radius(wl, ll, R):
    return sqrt((wl*ll/pi) + R**2)
final_r_prime = 0.08 # m
base_r = 0.05
last_distance = None
wl = 0.002  # width of leash in meters
ll = 1.0  # length in meters
signed_changed = False
change_length = 0.001
while True:
    
    r_prime = get_radius(wl, ll, base_r)
    new_distance = r_prime - final_r_prime
    if last_distance is not None:
        signed_changed = new_distance * last_distance < 0
    if signed_changed:
        change_length /= 2
    
    if last_distance is None or new_distance > 0:
        last_distance = new_distance
        ll -= change_length  # increase length by 1 cm
    else:
        ll += change_length  # decrease length by 1 cm
    
    if abs(new_distance)/final_r_prime < 0.01:  # stop when the difference is negligible
        break
print(f"Final leash length: {ll:.4f} m")

Final leash length: 5.9280 m


In [ ]:
get_radius(0.001, 5, 0.05)

0.06640677162084259

In [1]:
from multiprocessing import cpu_count
print(f"Number of CPU cores available: {cpu_count()}")

Number of CPU cores available: 32


In [1]:
import pyautogui
import time

def move_mouse_in_square(side_length, duration_per_side=0.5, num_squares=1):
    """
    Moves the mouse in a square pattern.

    Args:
        side_length (int): The length of each side of the square in pixels.
        duration_per_side (float): The time in seconds it takes to move along one side.
        num_squares (int): The number of times to complete the square pattern.
    """
    print(f"Moving mouse in a square pattern (side length: {side_length} pixels, {num_squares} times)...")
    print("Move your mouse to the starting position quickly after running.")
    time.sleep(3)  # Give user time to move mouse to a safe starting position

    current_x, current_y = pyautogui.position()
    print(f"Starting position: ({current_x}, {current_y})")

    for _ in range(num_squares):
        # Move right
        print(f"Moving right by {side_length} pixels...")
        pyautogui.move(side_length, 0, duration=duration_per_side)
        time.sleep(0.1) # Small pause between sides

        # Move down
        print(f"Moving down by {side_length} pixels...")
        pyautogui.move(0, side_length, duration=duration_per_side)
        time.sleep(0.1)

        # Move left
        print(f"Moving left by {side_length} pixels...")
        pyautogui.move(-side_length, 0, duration=duration_per_side)
        time.sleep(0.1)

        # Move up
        print(f"Moving up by {side_length} pixels...")
        pyautogui.move(0, -side_length, duration=duration_per_side)
        time.sleep(0.1)
    
    print("Finished square movement.")


while True:
    move_mouse_in_square(side_length=100, duration_per_side=5, num_squares=3)

Moving mouse in a square pattern (side length: 100 pixels, 3 times)...
Move your mouse to the starting position quickly after running.
Starting position: (839, 77)
Moving right by 100 pixels...
Moving down by 100 pixels...
Moving left by 100 pixels...
Moving up by 100 pixels...
Moving right by 100 pixels...
Moving down by 100 pixels...
Moving left by 100 pixels...
Moving up by 100 pixels...
Moving right by 100 pixels...
Moving down by 100 pixels...
Moving left by 100 pixels...
Moving up by 100 pixels...
Finished square movement.
Moving mouse in a square pattern (side length: 100 pixels, 3 times)...
Move your mouse to the starting position quickly after running.
Starting position: (837, 83)
Moving right by 100 pixels...
Moving down by 100 pixels...
Moving left by 100 pixels...
Moving up by 100 pixels...
Moving right by 100 pixels...
Moving down by 100 pixels...
Moving left by 100 pixels...
Moving up by 100 pixels...
Moving right by 100 pixels...
Moving down by 100 pixels...
Moving left 

FailSafeException: PyAutoGUI fail-safe triggered from mouse moving to a corner of the screen. To disable this fail-safe, set pyautogui.FAILSAFE to False. DISABLING FAIL-SAFE IS NOT RECOMMENDED.